# Задание 1

Для задачи линейного программирования: минимизировать $c^T x$ при условии $Ax = b$, 
$x ≥ 0$

a) (2б) Сформулируйте двойственную задачу и решите её (можно
использовать любой оптимизационный пакет)

b) (2б) С использованием двойственного решения найдите решение
исходной задачи (без использования оптимизационных пакетов)

*Решение пункта (a):*

Обозначим через $f(x) = c^T x, g(x) = -x, h(x) = Ax - b$. Тогда лагранжиан для нашей задачи будет выглядеть так: $L(x, \lambda, \mu) = f(x) + \lambda^T g(x) + \mu^T h(x) = c^T x - \lambda^T x + \mu^T (Ax - b) = (c^T - \lambda^T + \mu^T A)x - \mu^T b$.

$q(\lambda, \mu) = \inf\limits_{x \in \mathbb{R}^n} L(x, \lambda, \mu) = \inf\limits_{x \in \mathbb{R}^n} ((c^T - \lambda^T + \mu^T A)x - \mu^T b) = 
\begin{cases}
-\mu^T b , & c^T - \lambda^T + \mu^T A = 0 \\
-\infty, & c^T - \lambda^T + \mu^T A \neq 0
\end{cases}$

Двойственная задача будет следующей: максимизировать $-\mu^T b$ при условии $\lambda \geq 0$ и $c^T - \lambda^T + \mu^T A = 0$


In [1]:
import numpy as np
import cvxpy as cp

def solve_dual_task(A: np.array, b: np.array, c: np.array):
    '''
    Returns solution (lambda, mu)
    '''
    m, n = c.shape[0], A.shape[0]
    lambda_ = cp.Variable(m)
    mu = cp.Variable(n)
    constraints = [lambda_ >= 0, c - lambda_ + A.T @ mu == 0]
    obj = cp.Maximize(-mu.T @ b)
    cp.Problem(obj, constraints).solve()
    return (lambda_.value, mu.value)


*Решение пункта (b):*

Из условия Слейтера получаем, что оптимальное решение двойственной задачи даёт оптимальное решение прямой: $f(x^*) = q(\lambda^*, \mu^*) \Rightarrow c^T x^* = -{\mu^*}^T b$ и $c^T - {\lambda^*}^T + {\mu^*}^TA = 0, Ax^* = b, x^* \geq 0$, где $x^*$ -- решение прямой задачи, а $\lambda^*, \mu^*$ -- решение двойственной.

$c^T - {\lambda^*}^T + {\mu^*}^TA = 0, c^T x^* = -{\mu^*}^T b, Ax^* = b \Rightarrow {\lambda^*}^T x^* = 0$

Из условия дополняющей нежёсткости в теореме Каруша-Куна-Таккера следует, что если $\lambda^*_i \neq 0,$ то $x^*_i = 0$.
Получаем следующее решение прямой задачи: если $\lambda^*_i \neq 0$, то $x^*_i = 0$; иначе ищем координаты вектора $x_*$ из решения системы $Ax^* = b$

In [2]:
def solve_primal_task(A: np.array, b: np.array, c: np.array):
    '''
    Returns solution x*
    '''
    lambda_star, mu_star = solve_dual_task(A, b, c)
    index = np.isclose(lambda_star, 0)
    x_star = np.zeros(c.shape[0])
    A_ = A[:, index]
    x_star[index] = np.linalg.inv(A_.T @ A_) @ A_.T @ b
    return x_star

# Задание 2

Дан набор точек $x_1, . . . , x_m ∈ R^n$. Требуется найти шар минимального радиуса, содержащего все эти точки

*Решение:*

Можно минимизировать квадрат радиуса $r_{sqr}$ при условиях $r_{sqr} \geq 0$ и $(c - X[i])(c - X[i])^T \leq r_{sqr}$, где $с$ - координаты центра искомого шара, а $X$ - матрица $m$ x $n$ со строками-исходными точками

In [3]:
def get_center_and_radius(X: np.array):
    '''
    Args:
        X: (m, n) -- matrix with rows-points
    Returns:
        c: np.array (n,) -- center,
        r: float -- radius
    '''
    m, n = X.shape
    r_sqr = cp.Variable(1)
    c = cp.Variable(n)
    constraints = [np.sum(c**2) + np.sum(X[i, :]**2) - 2 * c @ X[i, :].reshape((-1, 1)) <= r_sqr for i in range(m)] + [r_sqr >= 0]
    obj = cp.Minimize(r_sqr)
    cp.Problem(obj, constraints).solve()
    return (c.value, np.sqrt(r_sqr.value))

X = np.array([[1, 0], [0, 1], [-1, 0]])
c, r = get_center_and_radius(X)
EPS = 1e-5
assert(np.linalg.norm(c) < EPS and abs(r - 1) < EPS)

# Задание 3

(4б) SVM: дан набор точек $x_1, . . . , x_m ∈ R^n$ и пометки $y_1, . . . , y_m ∈$ {$0$, $1$}. 

Требуется найти такой вектор $a$, что величина $\min \limits_{y_i = 0} a^T x_i − \max \limits_{y_i = 1} a^T x_i$ максимальна.

*Решение:*

Заметим, что в задаче мы ищем определённое направление, поэтому будем рассматривать векторы $a$ в единичном шаре.

Можно ввести две дополнительные переменные $min^*$ и $max^*$. И максимизировать $(min^* - max^*)$ при условиях:

1. $\forall i: y_i = 0: a^T x_i \geq min^*$
2. $\forall i: y_i = 1: a^T x_i \leq max^*$
3. $|a| \leq 1$

Причём, решив такую задачу, мы найдём именно минимум и максимум, так как иначе можно было бы уменьшить величину $(min^* - max^*)$ при фиксированном векторе $a$ из решения, подставив $min^* = \min \limits_{y_i = 1} a^T x_i$ и $max^* = \max \limits_{y_i = 0} a^T x_i$ (получили противоречие).

In [4]:
def solve_svm(X: np.array, y: np.array):
    '''
    Args:
        X: np.array (m, n) with rows-points
        y: np.array (m,) with labels from {0, 1}
    Returns:
        a: np.array (n,) - solution
    '''
    min_star = cp.Variable(1)
    max_star = cp.Variable(1)
    a = cp.Variable((1, X.shape[1]))
    constraints = [a @ row.T >= min_star for row in X[y == 0]] + [a @ row.T <= max_star for row in X[y == 1]] + [cp.sum_squares(a) <= 1]
    obj = cp.Maximize(min_star - max_star)
    cp.Problem(obj, constraints).solve()
    return a.value
